In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
from ola_models import * 
from ola_RNN import * 

import os, time, copy, math, re, json, pickle, random
import numpy as np
import pandas as pd

import torch, torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
%matplotlib inline 
import seaborn as sns

from functools import partial 

cuda_available = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda_available else "cpu")
print(f'''using device {device}''')

path = !pwd
path = path[0]
print(path)

using device cuda:0
/home/r2/Documents/RNNexp


In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [4]:
class Attention(nn.Module):
    def __init__(self,x_sz,h_sz,sql):
        super(Attention,self).__init__()
        self.x_sz = x_sz
        self.h_sz = h_sz 
        
        self.Wa   = nn.Linear(self.h_sz,self.h_sz)
        self.Wc   = nn.Linear(self.h_sz * 2,self.h_sz)
        self.Wend = nn.Linear(self.h_sz, self.x_sz)
        self.softmax = nn.LogSoftmax(dim=1)   
        
        self.Aenc = cuda(GRU(self.x_sz,self.h_sz))
        self.Adec = cuda(GRU(self.x_sz,self.h_sz))
        
    def get_hbar(self,X,Y,hidden=None):        
#         hbar = cuda(torch.zeros(X.shape[0],X.shape[1],self.h_sz))
        hbar = []
        print("in get_hbar")
        if hidden is None: hidden = self.initHidden(X.shape[0])
        for idx in range(X.shape[1]):
            x,y           = X[:,idx],Y[:,idx]
            x,y,hidden    = unpad(x,y,hidden)
            if x.shape[0] == 0: break               
            output, hidden = self.Aenc.forward(x,hidden)
            hbar.append(hidden)
        return hbar     
    
    def align(self,h,hbar,s):
        denominator = cuda(torch.zeros(h[0].shape))
        for s_p in range(len(hbar)):
            denominator = denominator + torch.exp(self.get_score(h[s],hbar[s_p])) 
        nominator = torch.exp(self.get_score(h[s],hbar[s]))
        return nominator/denominator
    
    def get_score(self,h_t,hbar):
        f""" dot: h_t * hbar_t """
        f""" concat: v_a * torch.tanh(self.score_Wa(torch.cat((h_t,hbar_t),1))) """
        f""" general """
        out = h_t * self.Wa(hbar)
        return out

    def initHidden(self,bs):
        return cuda(torch.zeros(bs,self.h_sz))

    def batch_forward(self,X,Y,hidden,loss_fn):

        hbar   = self.get_hbar(X,Y,hidden)      
        hidden = hbar[-1]
#         h      = cuda(torch.tensor(hbar.shape))
#         a      = cuda(torch.tensor(hbar.shape))
#         h_til  = cuda(torch.tensor(hbar.shape))
        h = []
        a = []
        h_til = []
        
        loss   = 0 
        accu   = 0 
        print("in batch_forward")
        with torch.autograd.set_detect_anomaly(True):
            for idx in range(X.shape[1]):
                x,y           = X[:,idx],Y[:,idx]
                x,y,hidden    = unpad(x,y,hidden)
                if x.shape[0] == 0: break   
                _,hidden = self.Adec.forward(x,hidden)
                h.append(hidden)
                a.append(self.align(h,hbar,idx))
                """paper: in global-attention at = ct (I'm pretty sure)"""
                h_til.append(torch.tanh(self.Wc(torch.cat((h[idx],a[idx]),1))))                
                output = self.Wend(h_til[idx])
                output = self.softmax(output)
                accu += get_accu(output,y)/x.shape[0]
                loss += loss_fn(output,y)
        return output,h[-1].detach(),loss/(idx+1),accu/(idx+1)

In [5]:
bs  = 2
sql = 30 
lr  = 0.0005 

sched = combine_scheds([0.15, 0.25, 0.2, 0.4], [sched_cos(0.0005, 0.0008), sched_cos(0.0008, 0.0005),sched_lin(0.0005,0.0005),sched_cos(0.0005,0.00005)]) 
data          = pp_trumpdata(path+"/data/trump/", [0.9,0.95], bs)
data.train_dl = TweetDataLoader(data,data.train.tweets,bs,sql,shuffle=True)
data.valid_dl = TweetDataLoader(data,data.valid.tweets,30,sql,shuffle=False)

In [6]:
model  = cuda(Attention(len(data.decoder), 150, sql))
opt    = optim.RMSprop(model.parameters(), lr)
learn  = Learner(model, nn.NLLLoss(), opt , data, lr=lr)

In [7]:
class PrintDataCallback(Callback):
    _order = 20
    
    def begin_fit(self,learn):
        super().begin_fit(learn)
        return True
    
    def begin_batch(self,xb,yb):
        print(xb.shape)
        print(yb.shape)
        for x,y in iter(zip(xb,yb)):            
            onehdecode(x,self.learn.data.decoder)
            ydecode(y,self.learn.data.decoder)
        return True

In [8]:
class CountCallback(Callback):
    _order = 16
        
    def __init__(self):            
        pass
    
    def begin_fit(self,learn):
        super().begin_fit(learn)
        return True

    def after_loss(self,loss):
        print(self.learn.n_iters)
        return True
    

In [9]:
cbs    = CallbackHandler([CounterCallback(1000),StatsCallback(),CountCallback(),PrintDataCallback()])

In [10]:
fit_rnn(1,learn,cbs)

torch.Size([2, 30, 433])
torch.Size([2, 30])
^"@TrumpNewMedia:  Get out &am
"@TrumpNewMedia:  Get out &amp
^“@limbaugh: 'Trump Has Change
“@limbaugh: 'Trump Has Changed
in get_hbar
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([422, 353], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([341, 341], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([114, 182], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([359,  83], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([331, 317], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([317,  87], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([134, 372], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([147, 331], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([218, 302], device='cuda:0')
tensor([0, 1], dev

tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([351,  17], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([153, 279], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([218, 197], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([114, 153], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([359,  83], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([331, 359], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([317, 218], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([134,  17], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([17, 56], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([ 83, 218], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([368,  87], device='cuda:0')
tensor([0, 1], device='

tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([372,  17], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([153, 341], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([ 17, 147], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([160, 218], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([ 83, 139], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([317, 160], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([134, 317], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([182, 372], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([218, 155], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([355, 184], device='cuda:0')
tensor([0, 1], device='cuda:0')
torch.Size([2, 150])
tensor([17, 68], device='cuda:0')
tensor([0, 1], device='

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
plot_list(learn.stats.train_loss[100::], 'train_loss')
plot_list(learn.stats.train_mva_loss[100::], 'moving_average_train_loss')
plot_list(learn.stats.valid_loss, 'valid loss')
plot_list(learn.stats.valid_accu, 'valid accuracy')